In [18]:
# !pip --version

In [17]:
# !pip install pytesseract
# !pip install Pillow
# !pip install opencv-python
# !pip install matplotlib

In [1]:
import pytesseract as tess
tess.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
from cv2 import cv2
from PIL import Image
import matplotlib.pyplot as plt



In [4]:
im = cv2.imread('page_1_cropped.jpg')

gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (9,9), 0)
# thresh2 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)
thresh2 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Dilate to combine adjacent text contours
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
dilate = cv2.dilate(thresh2, kernel, iterations=4)

# Find contours, highlight text areas, and extract ROIs
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
# cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

line_items_coordinates = []
for c in cnts:
    area = cv2.contourArea(c)
    x,y,w,h = cv2.boundingRect(c)

    weight = 3900
    weight2 = 20
    # print("x,y,w,h",x,y,w,h)
    if area > 10000:
            image = cv2.rectangle(im, (x-weight2,y-weight2), (weight, y+h), color=(255,0,255), thickness=1)
            line_items_coordinates.append([(x-weight2,y-weight2), (weight, y+h)])
    
cv2.imwrite(r'xregionified4.jpg',image)
#-----------
def show_images(titles, images):
    # titles = ['dilate']
    # images = [dilate]
    _range = len(images)

    for i in range(_range):
        plt.subplot(1, 3, i+1), plt.imshow(images[i], 'gray')
        plt.title(titles[i])
        plt.xticks([]),plt.yticks([])

    plt.show()
#-----------

# ################### OCR Part
# # load the original image
# image = cv2.imread('page_1_cropped.jpg')
# image_copy = image
# # for i,cord in reversed(list(enumerate(line_items_coordinates))):
# for i,cord in reversed(list(enumerate(line_items_coordinates))):
#     # get co-ordinates to crop the image
#     c = line_items_coordinates[i]

#     #Get area to crop ROIs
#     length = c[1][0] - c[0][0]
#     height = c[1][1] - c[0][1]
#     area = length * height

#     if (area >= 200000 ):
#     # cropping image img = image[y0:y1, x0:x1]
#         img = image[c[0][1]:c[1][1], c[0][0]:c[1][0]]    

#     # convert the image to black and white for better OCR
#     gray2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     ret1,thresh1 = cv2.threshold(gray2,120,255,cv2.THRESH_BINARY)
#     thresh2 = cv2.threshold(gray2, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#     bilf = cv2.bilateralFilter(gray2,9,75,75)

#     # titles = ['thresh1','thresh2', 'bilf']
#     # images = [thresh1, thresh2, bilf]
#     # show_images(titles, images)

    
#     # pytesseract image to string to get results
#     text = str(tess.image_to_string(thresh2, lang='sin', config='--psm 6'))
#     print(text)

TEST

In [16]:
###### OCR Part
# load the original image
image = cv2.imread('page_1_cropped.jpg')
img = image

gray2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret1,thresh1 = cv2.threshold(gray2,120,255,cv2.THRESH_BINARY)
thresh2 = cv2.threshold(gray2, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

bilf = cv2.bilateralFilter(gray2,9,75,75)

    
# pytesseract image to string to get results
text = str(tess.image_to_string(thresh2, lang='sin+eng', config='--psm 6'))
print(text)

| 1]49/ 1 වැල්මිල්ල ආරච්චිගේ දොන උදය ප්‍රියශාන්ත කුමාර 8211731671" පි ]
පනාකඩු ලියනගේ රුපචතී 555091508V ගැ 2
වැල්මිල්ල ආරච්චිගේ දොන්‌ සුදර්ශනී රූපිකා කුමාරි 808213796V ගැ 3

[1]49/2 මලේ පතිරණගේ නිශාන්ත මොහන්‌ වික්‍රමසිංහ 662141682V පි 4
රත්නායක මුදියන්සේලාගේ යමුනා පියසීලි 637963589V ගැ ට්‌
රත්නායක
මලේ පතිරණගේ නිමන්තිකා මධුභාෂිණ වික්‍රමසිංහ 935660904V ගැ 6
මලේ පතිරණගේ විශ්මිකා නිර්මාණි වික්‍රමසිංහ 975800288V ගැ 7



In [39]:
import re
from collections import namedtuple
import pandas as pd


In [45]:
line_test ="""[1]49/1 වැල්මිල්ල ආරච්චිගේ දොන උදය ප්‍රියශාන්ත කුමාර 821173167V     ගැ 1
| 1]49/2 මපතිරණගේ නිශාන්ත මොහන වික්‍රමසිංහ 662141682V පි 4"""
# print(re.match(r'([\[|]\W*\d+[\]|]\d+\/?\d+)\s+', line_test))
heading  = r'[1]49/1 වැල්මිල්ල ආරච්චිගේ දොන උදය ප්‍රියශාන්ත කුමාර 821173167V     ගැ 1'
print(re.match(r'([\[|]\W*\d+[\]|]\d+\/?\d+)(.*\s+)(\d+[VvXx]?)\s*(\W+)\s*.*(\d+)', line_test).group(2))
# print(re.match(r'[^\W\d_]', line_test).group(2))
# print(line_re.findall(line_test))

arr = re.match(r'([\[|]\W*\d+[\]|]\d+\/?\d+)(.*\s+)(\d+[VvXx]?)\s*(\W+)\s*.*(\d+)', line_test).group(2)

# print(arr)
df = pd.DataFrame(eval(arr))

df.to_excel('tester1.csv', header=True, index=False, encoding='utf-8')

 වැල්මිල්ල ආරච්චිගේ දොන උදය ප්‍රියශාන්ත කුමාර 821173167V     ගැ 


SyntaxError: invalid syntax (<string>, line 1)

In [36]:
# Writing to a csv
Line = namedtuple('Line', 'House_No Name NIC Gender Index')

line_test ="[1]49/1 වැල්මිල්ල ආරච්චිගේ දොන උදය ප්‍රියශාන්ත කුමාර 821173167V     ගැ 1"

line_re = re.compile(r'([\[|]\W*\d+[\]|]\d+\/?\d+)\s+(\W+)(\d+[VvXx]?)\s*(\W+)\s*(\d+)',re.UNICODE)

lines = []
if re.match(r'([\[|]\W*\d+[\]|]\d+\/?\d+)(.*\s+)(\d+[VvXx]?)\s*(\W+)\s*.*(\d+)', line_test).group(2):
    items = line_test.split()
    print(len(items))
    print(items[1:])
    lines.append(Line(*items))
else:
    print('nope')

df = pd.DataFrame(lines)

# df.to_csv('tester.csv', header=True, index=False, encoding='utf-8')

10
['වැල්මිල්ල', 'ආරච්චිගේ', 'දොන', 'උදය', 'ප්\u200dරියශාන්ත', 'කුමාර', '821173167V', 'ගැ', '1']


TypeError: __new__() takes 6 positional arguments but 11 were given

In [ ]:
df.info()
df['Name'] = df['Name'].astype('unicode')
df['Name'] = df['Name'].apply(lambda x: x.encode('utf-8').strip())
df['Name'] = df['Name'].map(lambda x: x.encode('unicode-escape').decode('utf-16'))
df['Name'] = df['Name'].map(lambda x: x.encode('unicode-escape'))
print(df['Name'])